In [1]:
import tensorflow as tf 
import os

The NEU-CLS dataset has 1800 grayscale images of steel surface defects (200×200 pixels) in six classes: rolled-in scale, patches, crazing, pitted surface, inclusion, and scratches.

The dataset is organized into subdirectories per class in separate train/images and validation/images directories. Example: training images might be in .../train/images/scratches/ and similarly for the other classes. 

I would define my steps for training the classification model as follows:

Part 1: Data Processing

    1a. Proper loading of NEU data
    1b. Augment training set and normalize both sets for unified comparison
  
Part 2: Model Creation and Training Parameters

    2a. Defining Convolution Neural Net Model
    2b. Establishing Callbacks to stop training when optimization met.
  
Part 3: Training and Selecting Optimal Model

    3a. Training CNN with established parameters, and saving best model into 'SmartSSD/best_model.keras'.

# Part 1: Data Processing

## 1a. Proper loading of NEU data.

For reproducability I am using python's **os** package to establish given user's current working directory. Then joining with location of my train & validation data.

In [2]:
# user dir
current_dir = os.getcwd()

# known filepaths
train_dir = os.path.join(current_dir, "NEU-CLS", "train", "images")
val_dir   = os.path.join(current_dir, "NEU-CLS", "validation", "images")

# unified variables
image_size = (128, 128)
batch_size = 32
epochs = 10
num_classes = 6

## 1b. Augment training set and normalize both sets for unified comparison

For loading my images and labels I am using Keras package to read directories of images. My code uses TF's *ImageDataGenerator()* instead of the basic *image_dataset_from_directory()* for integrated data augmentation such as normalization (*rescale=1. / 255*) and transformations (to the training data only). By augmenting I am expanding the size of the dataset so my model has more images to train on without having to gather any new data. 

- Normalization: All images rescaled to be in range(0,1) by dividing by 225 and resizing to 128x128 resolution. *color_mode='greyscale'* specified so each image has one channel.

- Augmentation: My transformations to the training data includes rotation and reflection (flip), as well as brightness transformation. Limiting range of augmentation to at most 20% for ensuring the image remains usable. No transformations/augmentations made to validation data to avoid leaking val information.

In [3]:
# Augmenting training images by rotating flipping and altering brightness
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255,
                                                                rotation_range=20, # random rotations
                                                                horizontal_flip=True, # random horizontal flips
                                                                brightness_range=(0.8, 1.2)) # random brightness

# just normalizing in the case of val images
val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255) 

# actually generating the image defined from my ImageDataGenerator() function
train_gen = train_datagen.flow_from_directory(train_dir,
                                              target_size=image_size,
                                              color_mode='grayscale',
                                              batch_size=batch_size,
                                              class_mode='categorical')

val_gen = val_datagen.flow_from_directory(val_dir,
                                          target_size=image_size,
                                          color_mode='grayscale',
                                          batch_size=batch_size,
                                          class_mode='categorical')

Found 1440 images belonging to 6 classes.
Found 360 images belonging to 6 classes.


# Part 2. Defining Convolution Neural Net Model

## 2a. I will create a basic **Convolutional Neural Network (CNN)** consisting of three blocks and a classification head. 

Block 1, 2, 3: 

> Layer 1. Convolution layer with **ReLu** activation function. *Conv2D()*
>
> Layer 2. Establishing limitations to my batch pool_size. *MaxPooling2D()*
>
> Layer 3. *Dropout()* layer limiting overfitting (also normalizing).

Classification Head:

> Layer 1: *Flatten()* layer to  the feature maps.
>
> Layer 2: *Dense()* layer using **ReLU** as my activation.
> 
> Layer 3: *Dropout()* layer to limit overfitting model.
>
> Layer 4: *Dense()* layer but now using number of types of defects (classes), thus switch to **softmax** activation.

In [4]:
model = tf.keras.models.Sequential([
    # Block 1
    tf.keras.layers.Conv2D(32, (3, 3), activation= 'relu', input_shape= (128, 128, 1)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Dropout(0.25),
    
    # Block 2
    tf.keras.layers.Conv2D(64, (3, 3), activation= 'relu'),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Dropout(0.25),
    
    # Block 3
    tf.keras.layers.Conv2D(128, (3, 3), activation= 'relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Dropout(0.25),
    
    # Classification head
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation= 'relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(num_classes, activation= 'softmax')
])

c:\ProgramData\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Finally, my last block will compile my established CNN. Since there are multiple different classifications possible for image, I will compile model with **Adam** as my optimizer and measure by categorical cross-entropy loss.

In [5]:
model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 126, 126, 32)   │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     3,211,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 6)              │           774 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,304,838 (12.61 MB)

 Trainable params: 3,304,838 (12.61 MB)

 Non-trainable params: 0 (0.00 B)

## 2b. Establishing Callbacks to stop training when optimization met.

I will train my model with the entire dataset (*epochs=*) 10 times. While fitting the model using *model.fit()* in Part 3, Keras autoprints training and validation accuracy/loss for every epoch. I will set *callbacks=* 'EarlyStopping' and 'ModelCheckpoint' in *model.fit()* to accomplish the following:

- EarlyStopping: While monitoring training loss, I will stop training if model performance does not improve for 3 epochs. For instance, if epoch 2 has val_loss of 0.1 (hypothetically), and epochs 3, 4, 5 have val_loss greater than 0.1, training should stop after epoch 5, and the model weights from epoch 2 should be loaded.

- ModelCheckpoint: I save model weights to file **best_model.keras** whenever I get improvement of performance on the validation set by setting *save_best_only=* True.

In [6]:
earlyStop = tf.keras.callbacks.EarlyStopping(monitor= 'val_loss',
                                             patience= 3, # test for improvement in next 3, if none stop and load old
                                             restore_best_weights= True,
                                             verbose= 1)

# after stopping loading weights from old epoch (the one with the lowest val loss)
modelCheckpoint = tf.keras.callbacks.ModelCheckpoint('best_model.keras',
                                                     monitor= 'val_loss',
                                                     save_best_only= True,
                                                     verbose= 1)

callbacks = [earlyStop, modelCheckpoint]

# Part 3: Training and Selecting Optimal Model

## 3a. Training and saving my best performing model to 'SmartSSD/best_model.keras'.

In order to use the trained model for classification on new data, I will save it to my directory in file **best_model.keras**. 

In [7]:
history = model.fit(train_gen,
                    epochs= epochs,
                    validation_data= val_gen,
                    callbacks= callbacks,
                    verbose= 1)

Epoch 1/10


c:\ProgramData\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step - accuracy: 0.1654 - loss: 1.9149
Epoch 1: val_loss improved from inf to 1.77510, saving model to best_model.keras
45/45 ━━━━━━━━━━━━━━━━━━━━ 4s 82ms/step - accuracy: 0.1656 - loss: 1.9130 - val_accuracy: 0.1667 - val_loss: 1.7751
Epoch 2/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step - accuracy: 0.2282 - loss: 1.7270
Epoch 2: val_loss improved from 1.77510 to 1.72828, saving model to best_model.keras
45/45 ━━━━━━━━━━━━━━━━━━━━ 4s 81ms/step - accuracy: 0.2284 - loss: 1.7266 - val_accuracy: 0.4278 - val_loss: 1.7283
Epoch 3/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step - accuracy: 0.3529 - loss: 1.6059
Epoch 3: val_loss improved from 1.72828 to 1.70060, saving model to best_model.keras
45/45 ━━━━━━━━━━━━━━━━━━━━ 4s 80ms/step - accuracy: 0.3532 - loss: 1.6052 - val_accuracy: 0.3000 - val_loss: 1.7006
Epoch 4/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step - accuracy: 0.4321 - loss: 1.4518
Epoch 4: val_loss improved from 1.70060 to 1.38632, saving model to best

Even though my models val set accuracy is comparable in epochs 4 and 7, we can see that validation loss is significantly greater for epoch 7. Thus I decided to retrain with a higher early stop patience. Now I will continue testing for 5 values after find local maxima. I will still monitor validation loss.

In [8]:
earlyStop = tf.keras.callbacks.EarlyStopping(monitor= 'val_loss',
                                             patience= 5, # test for improvement in next 3, if none stop and load old
                                             restore_best_weights= True,
                                             verbose= 1
                                             )

# after stopping loading weights from old epoch (the one with the lowest val loss)
modelCheckpoint = tf.keras.callbacks.ModelCheckpoint('best_model.keras',
                                                     monitor= 'val_loss',
                                                     save_best_only= True,
                                                     verbose= 1
                                                     )

callbacks = [earlyStop, modelCheckpoint]

history = model.fit(train_gen,
                    epochs= epochs,
                    validation_data= val_gen,
                    callbacks= callbacks,
                    verbose= 1
                    )

Epoch 1/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step - accuracy: 0.8048 - loss: 0.5332
Epoch 1: val_loss improved from inf to 0.90984, saving model to best_model.keras
45/45 ━━━━━━━━━━━━━━━━━━━━ 4s 83ms/step - accuracy: 0.8046 - loss: 0.5339 - val_accuracy: 0.6583 - val_loss: 0.9098
Epoch 2/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step - accuracy: 0.8000 - loss: 0.5639
Epoch 2: val_loss improved from 0.90984 to 0.77774, saving model to best_model.keras
45/45 ━━━━━━━━━━━━━━━━━━━━ 4s 81ms/step - accuracy: 0.7995 - loss: 0.5659 - val_accuracy: 0.7306 - val_loss: 0.7777
Epoch 3/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step - accuracy: 0.7366 - loss: 0.8488
Epoch 3: val_loss improved from 0.77774 to 0.70642, saving model to best_model.keras
45/45 ━━━━━━━━━━━━━━━━━━━━ 4s 81ms/step - accuracy: 0.7379 - loss: 0.8442 - val_accuracy: 0.7056 - val_loss: 0.7064
Epoch 4/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step - accuracy: 0.8525 - loss: 0.4533
Epoch 4: val_loss did not improve from 0.70642
45/45 ━━━━━━━━

Our new validation loss is considerably better (0.4458) and now selects epoch 7 as the optimal. Additionally, we can see significant improvement in validation set classification accuracy (0.83) compared to our previous selection of epoch 4 (0.64).